In [1]:
## setup the environment
# !wget -O prepare_corpus.py https://raw.githubusercontent.com/tianyizz/project-reddit_watson/master/prepare_corpus.py
# !wget -O urlmarker.py https://raw.githubusercontent.com/tianyizz/project-reddit_watson/master/urlmarker.py
# !pip install watson_developer_cloud
!pip install ibm_watson

     |████████████████████████████████| 368kB 17.7MB/s eta 0:00:01
     |████████████████████████████████| 204kB 30.5MB/s eta 0:00:01
  Stored in directory: /home/dsxuser/.cache/pip/wheels/b7/10/b4/509307276e9d558a6b06d8e69f8faf6cfbdf2d1b83abb6b8d4
  Stored in directory: /home/dsxuser/.cache/pip/wheels/6a/42/50/f96888116b329578304f9dda4693cef6f3e76e18272d22cb6c
Successfully built ibm-watson ibm-cloud-sdk-core


In [2]:
# import types
import pandas as pd
import numpy as np
from botocore.client import Config
import ibm_boto3
# import nltk
# nltk.download("stopwords")
import json
# from prepare_corpus import *
import csv
# from watson_developer_cloud import NaturalLanguageUnderstandingV1
# import watson_developer_cloud.natural_language_understanding_v1 as ai
import logging
logging.basicConfig(filename='error.log', level=logging.ERROR)  # This defines the name of the error log file
from concurrent.futures import ThreadPoolExecutor
# from tqdm import tqdm
import time
from datetime import datetime
from ibm_watson import NaturalLanguageUnderstandingV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from ibm_watson.natural_language_understanding_v1 import Features, EmotionOptions, SentimentOptions

def __iter__(self): return 0

In [3]:
# @hidden_cell
# The project token is an authorization token that is used to access project resources like data sources, connections, and used by platform APIs.
from project_lib import Project
project = Project(project_id='42dd98d3-0b00-4676-809d-015d08bf8d84', project_access_token='p-9249755dbd201d74774e21b20c1c46e09b1e31c3')
pc = project.project_context

# Import all tweets from IBM Watson Project

In [4]:
project.get_file('all_tweets_with_stratify_groups-2020_02_13.csv')

[{'file_content': <_io.BytesIO at 0x7f602286c678>,
  'data_file_title': 'all_tweets_with_stratify_groups-2020_02_13.csv'},
 {'file_content': <_io.BytesIO at 0x7f602286c6d0>,
  'data_file_title': 'data_profile_8729b793-18fa-464f-b2d4-49b16e5fc696'}]

In [5]:
df = pd.read_csv(project.get_file('all_tweets_with_stratify_groups-2020_02_13.csv')[0]['file_content'])

In [6]:
# look at the datadd
df

,tweet_id,is_ira,date,year,month,twitter_account,text,stratify
0,671478900036210690,1,2015-12-01 00:00:00,2015,12,nyc_everyday,NYC seeks new vendor to operate McCarren Park ...,15
1,671479367487217665,1,2015-12-01 00:02:00,2015,12,nyc_everyday,Celebrity honeymoon photos #showbiz,15
2,671482584908488704,1,2015-12-01 00:14:00,2015,12,nyc_everyday,The Neediest Cases: A Harlem Man’s First Steps...,15
3,671495873369518080,1,2015-12-01 01:07:00,2015,12,nyc_everyday,#politics Paris climate talks creates big carb...,15
4,671500883214000129,1,2015-12-01 01:27:00,2015,12,nyc_everyday,Syrian Family of 7 Is Quickly Settled in New J...,15
5,671503389717479424,1,2015-12-01 01:37:00,2015,12,nyc_everyday,#politics Ala. lawmaker warns of non-American ...,15
6,671507480585760768,1,2015-12-01 01:53:00,2015,12,nyc_everyday,"Facing Giants, Jets Focus on Playoffs, Not Riv...",15
7,671510979969069056,1,2015-12-01 02:07:00,2015,12,nyc_everyday,Man pleads not guilty for molesting sleeping w...,15
8,671514813856632832,1,2015-12-01 02:23:00,2015,12,nyc_everyday,'DWTS' pro says Church of Scientology spied on...,15
9,671518950065233921,1,2015-12-01 02:39:00,2015,12,nyc_everyday,Woodrow Wilson’s Legacy Gets Complicated #news,15


# TODO:

~1. combine the two sets into a single df called `df` keeping id column, text, and a column for whether it was control or IRA.~
2. modify below code to work with `df`

~3. change save location to google drive rather than ibm cloud~
4. change string names to backup data in appropriate cloud object store bucket use "russiantrolls-donotdelete-pr-f0fvpzx7cjymub" bucket
5. Update api keys
6. Remove unnecessary code
7. Add comments

# Set up IBM NLU credentials

In [7]:
# note: test using lite-tier service 
# # use standard tier for final run

authenticator = IAMAuthenticator("uBuJNuYBAU5iKzmQ-8eigmM3AL2tT0UcNCcbbV1lOxa4")
service = NaturalLanguageUnderstandingV1(
    version='2019-07-12',
    authenticator=authenticator
)

# Use Watson NLU

### Define functions to make NLU call and process results

In [8]:
# get response from NLU
def get_response(x):
    try:
        resp = service.analyze(text=x.text,
                                   features=Features(
                                       sentiment=SentimentOptions(),
                                       emotion=EmotionOptions())
                                   ).get_result()
    except Exception as e:
        resp = None
        print("Something went wrong.... Error: ", e)
        logging.error(e)  # This should log the error messages to an error.log file

    return resp

# prepare the response for entry into a dataframe
def condition_response(resp):
    if resp is None:        
        cr = dict(nlu_status = "fail",
                emotion_anger = None,
                emotion_disgust = None,
                emotion_fear = None,
                emotion_joy = None,
                emotion_sadness = None,
                sentiment_label = None,
                sentiment_score = None,
                language = None,
                n_char = None)
    else:
        emotions = None if resp.get('emotion') is None else resp['emotion']['document']['emotion']
        sentiments = None if resp.get('sentiment') is None else resp['sentiment']['document']
        
        cr = dict(nlu_status = "pass",
                emotion_anger =  None if emotions is None else emotions['anger'],
                emotion_disgust = None if emotions is None else emotions['disgust'],
                emotion_fear = None if emotions is None else emotions['fear'],
                emotion_joy = None if emotions is None else emotions['joy'],
                emotion_sadness = None if emotions is None else emotions['sadness'],
                sentiment_label = None if sentiments is None else sentiments['label'],
                sentiment_score = None if sentiments is None else sentiments['score'],
                language = resp.get('language'),
                n_char = None if resp.get('usage') is None else resp['usage']['text_characters'])
        
    return cr

# process an individual row
def process_row(x):
    rowIndex = x.name
    
    cr = condition_response(get_response(x))
    cr['tweet_id'] = x['tweet_id']
    
    return cr

In [9]:
# define a function to do this nicely:
def get_results_multi_thread(df, nthread=30):
    
    # get an iterable for the rows of the dataframe
    rows = df.iterrows()
    nrows = df.shape[0]

    # wrapper function to handle extra index in iterable
    def process_row_no_idx(row):
        return process_row(row[1])
    
    # send rows to Watson NLU asynchronously
    with ThreadPoolExecutor(max_workers=nthread) as executor:
        # list(tqdm(executor.map(f, my_iter), total=len(my_iter)))
        futures = executor.map(process_row_no_idx, rows)

    # turn above iterator into list of results
    list_results = []
    for r in futures:
        list_results.append(r)

    # turn the list into a dataframe
    df_results = pd.DataFrame(list_results, columns=['tweet_id', 
                                                       'nlu_status', 
                                                       'language',
                                                       'n_char', 
                                                       'sentiment_label', 
                                                       'sentiment_score', 
                                                       'emotion_anger', 
                                                       'emotion_disgust', 
                                                       'emotion_fear', 
                                                       'emotion_joy', 
                                                       'emotion_sadness'])
    
    # return the results
    return df_results

In [10]:
get_results_multi_thread(df.sample(100), nthread=20)

,tweet_id,nlu_status,language,n_char,sentiment_label,sentiment_score,emotion_anger,emotion_disgust,emotion_fear,emotion_joy,emotion_sadness
0,768192615858790401,pass,en,96,neutral,0.000000,0.046866,0.018194,0.051937,0.531739,0.081548
1,765334280792862720,pass,en,125,positive,0.691165,0.005296,0.006325,0.001833,0.957638,0.024180
2,718710175587311616,pass,en,90,neutral,0.000000,0.085940,0.124879,0.084747,0.079928,0.265842
3,799931051888050176,pass,en,123,neutral,0.000000,0.206652,0.189667,0.154039,0.065423,0.544626
4,742500735825391616,pass,en,126,neutral,0.000000,0.042179,0.348853,0.005410,0.481292,0.201966
5,787057415342567424,pass,en,124,neutral,0.000000,0.042296,0.230857,0.159557,0.035611,0.368430
6,757986413409337344,pass,en,104,neutral,0.000000,0.212058,0.419298,0.188090,0.041959,0.379622
7,749081862018961408,pass,en,129,positive,0.876465,0.006236,0.002901,0.047690,0.935865,0.012463
8,798594757844996096,pass,en,158,neutral,0.000000,0.219004,0.296672,0.121221,0.171220,0.327102
9,792447728844468224,pass,en,87,neutral,0.000000,0.273397,0.076512,0.085317,0.101162,0.212300


In [ ]:
#TODO: change COS bucket/names

num_splits = 200
#df_test = df.sample(3000, replace=False)
#df_split = np.array_split(df_test, num_splits)
df_split = np.array_split(df, num_splits)

now = datetime.now()

split_idx = 0
loop_times = []
cos_res = []

total_start = time.time()

for d in df_split:
    start_time = time.time()
    # progress
    print("Processing %i rows for split %i of %i"  % (d.shape[0], split_idx+1, num_splits))
    
    try:
        # get nlu results
        d_res = get_results_multi_thread(d, nthread=40)

        # write to file
        filename = 'df_trolls_'+ now.strftime("%Y-%m-%d--%H-%M-%S") + str(split_idx).zfill(4) + '.csv' 
        project.save_data(filename, data=d_res.to_csv(index=False), overwrite=False)
    except Exception as e:
        print("Failed to process split %i. Exception: %s" % (split_idx, e))
        filename_err = 'err_df_trolls_' + now.strftime("%Y-%m-%d--%H-%M-%S") + str(split_idx).zfill(4) + '.csv' 
        print("Saving failed split data to cos bucket. Key: %s" % filename_err)
        
        #project.save_data(filename_err, data=d.to_csv(index=False), set_project_asset=True, overwrite=True)
        project.save_data(filename_err, data=d.to_csv(index=False), overwrite=False)
    
    duration = time.time() - start_time
    total_time = time.time() - total_start
    loop_times.append(duration)
    avg_loop_time = sum(loop_times) / len(loop_times)
    est_remain = (num_splits - split_idx - 1)*avg_loop_time / 60
    
    # Show progress
    print(" --Iter. time: %0.1f seconds\n" % duration,
          "--Avg. iter. time: %0.1f seconds\n" % avg_loop_time,
          "--Est. time remaining: %0.1f minutes\n" % est_remain,
         "--Total time: %0.1f seconds\n" % total_time)
    
    split_idx+=1


Processing 6405 rows for split 1 of 200
Something went wrong.... Error:  Error: unsupported text language: da, Code: 400 , X-global-transaction-id: 2c832a7a97f42be433134734a662f9af
Something went wrong.... Error:  Error: unsupported text language: da, Code: 400 , X-global-transaction-id: 40269c74e863533020fa60fcfc276482
Something went wrong.... Error:  Error: unsupported text language: unknown, Code: 400 , X-global-transaction-id: a1e30c67d2e9ce0fe7e4249345009499
Something went wrong.... Error:  Error: not enough text for language id, Code: 422 , X-global-transaction-id: 6afd12cf9d0cdf353c3436e559936679
Something went wrong.... Error:  Error: unsupported text language: da, Code: 400 , X-global-transaction-id: c90591c152a2782ea10405ef1df989fb
Something went wrong.... Error:  Error: unsupported text language: da, Code: 400 , X-global-transaction-id: f80d58578f667470af5e6fad019a17b3
Something went wrong.... Error:  Error: not enough text for language id, Code: 422 , X-global-transaction-i